In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
# import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
print(SEED)
# torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)

0


In [3]:
# missingness_levels = [0.1, 0.2, 0.3]

# for level in missingness_levels:
# # for level in missingness_levels[:1]:
#     train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_missingness_{level}.csv')
#     train_df = train_df.fillna(-1)
# #     print(len(train_df))
#     X_train = train_df.iloc[:, 0:-1]
#     y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train)
# #     print(X_train.shape, y_train.shape)
    
#     for steps in [int(15e6), int(12e6), int(10e6)]:
# #     for steps in [int(2e3)]:
#         dqn_model = utils.stable_dueling_ddqn(X_train, y_train, steps, True, 
#         f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_ddqn_pr_missingness_{level}_{steps}')

In [4]:
# sizes = [0.5]
# for size in sizes:
#     train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_basic_{size}_seed_{SEED}.csv')
#     train_df = train_df.fillna(-1)
#     X_train = train_df.iloc[:, 0:-1]
#     y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train)
#     for steps in [int(15e6)]:
#         dqn_model = utils.stable_vanilla_dqn(X_train, y_train, steps, True, 
#             f'../../../anemia_ml4hc/models/sb/new_different_training_sizes/dqn_pr_train_set_{size}_{SEED}_{steps}')

In [5]:
# noise_levels = [0.1, 0.2, 0.3]

# for level in noise_levels:
# # for level in noise_levels[:1]:
#     train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_noisiness_{level}.csv')
# #     print(len(train_df))
#     X_train = train_df.iloc[:, 0:-1]
#     y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train)
#     print(X_train.shape, y_train.shape)
    
#     for steps in [int(15e6), int(12e6), int(10e6)]:
# #     for steps in [int(2e3)]:
#         dqn_model = utils.stable_vanilla_dqn(X_train, y_train, steps, True, 
#                     f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/dqn_pr_noisiness_{level}_{steps}')#,

#### The Data

In [6]:
level = 0.3
train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_noisiness_0.2_missingness_{level}.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,12.570316,461.686352,-1.000000,2.195594,274.677370,98.593975,-1.0,3.824874,1,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,37.710949,-1.000000,5.0
1,6.025877,65.947255,5.269617,-1.000000,-1.000000,78.152338,-1.0,2.313127,0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,7.0
2,7.971342,-1.000000,3.031102,0.000000,391.557192,102.908562,-1.0,2.323813,1,1.750675,78.906129,-1.0,-1.000000,-1.0,-1.000000,23.914025,-1.000000,2.0
3,8.648926,395.552568,-1.000000,-1.000000,-1.000000,97.030821,-1.0,2.674076,0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,25.946777,-1.000000,6.0
4,16.785024,410.993164,3.460755,0.969443,-1.000000,80.127153,-1.0,-1.000000,1,0.290346,89.925567,-1.0,33.165717,-1.0,127.932614,50.355073,56.439819,0.0


In [7]:
len(train_df)

50400

In [8]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [9]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50400, 17), (14000, 17), (50400,), (14000,))

#### Training 

In [10]:
# %%time
# timesteps = int(1e6)
# dqn_model = utils.stable_dqn(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [11]:
# missingness
# if level==0.1:
#     steps = int(15e6)
# elif level==0.2:
#     steps = int(10e6)
# elif level==0.3:
#     steps = int(12e6)
# else:
#     print('UNKNOWN LEVEL')
# level, steps

In [12]:
# #noisiness
# if level==0.1:
#     steps = int(10e6)
# elif level==0.2:
#     steps = int(12e6)
# elif level==0.3:
#     steps = int(10e6)
# else:
#     print('UNKNOWN LEVEL')
# level, steps
steps = int(15e6)
level, steps

(0.3, 15000000)

In [ ]:
# for steps in [int(50e6)]:
#     dqn_model = utils.stable_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/complete_models/dueling_ddqn_{steps}',
#                                  checkpoint_folder = '../../../anemia_ml4hc/models/sb/dueling_double_dqn',
#                                  checkpoint_prefix='dddqn_basic')
    
# for steps in [int(15e6)]:
# # for steps in [int(2e3)]:
#     dqn_model = utils.stable_double_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/sb/seed_{SEED}/ddqn_pr_{steps}')#,
#                                  steps = int(15e6)
# dqn_model = utils.stable_dueling_dqn(X_train, y_train, steps, True, 
# f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_dqn_pr_noisiness_{level}_{steps}')

dqn_model = utils.stable_dueling_ddqn(X_train, y_train, steps, True, 
f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_ddqn_pr_missingness_{level}_{steps}')

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWa

using just stable baselines (not 3) - dueling double dqn
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.


Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning:

stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.



--------------------------------------
| % time spent exploring  | 98       |
| episodes                | 10000    |
| mean 100 episode reward | -0.9     |
| steps                   | 26494    |
| success rate            | 0.08     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 96       |
| episodes                | 20000    |
| mean 100 episode reward | -0.7     |
| steps                   | 53401    |
| success rate            | 0.14     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 94       |
| episodes                | 30000    |
| mean 100 episode reward | -0.8     |
| steps                   | 80770    |
| success rate            | 0.1      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 93       |
| episodes                | 40000    |
| mean 100 episode reward | -0.7     |
| steps                  

--------------------------------------
| % time spent exploring  | 40       |
| episodes                | 320000   |
| mean 100 episode reward | -0.9     |
| steps                   | 942230   |
| success rate            | 0.08     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 38       |
| episodes                | 330000   |
| mean 100 episode reward | -0.9     |
| steps                   | 972456   |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 36       |
| episodes                | 340000   |
| mean 100 episode reward | -0.9     |
| steps                   | 1002293  |
| success rate            | 0.16     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 34       |
| episodes                | 350000   |
| mean 100 episode reward | -0.9     |
| steps                  

...

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2800000  |
| mean 100 episode reward | -0.3     |
| steps                   | 9583286  |
| success rate            | 0.42     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2810000  |
| mean 100 episode reward | -0.4     |
| steps                   | 9621561  |
| success rate            | 0.42     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2820000  |
| mean 100 episode reward | -0.3     |
| steps                   | 9663068  |
| success rate            | 0.41     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2830000  |
| mean 100 episode reward | -0.2     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3110000  |
| mean 100 episode reward | -0.1     |
| steps                   | 10755891 |
| success rate            | 0.51     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3120000  |
| mean 100 episode reward | -0.1     |
| steps                   | 10793501 |
| success rate            | 0.53     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3130000  |
| mean 100 episode reward | -0.2     |
| steps                   | 10830535 |
| success rate            | 0.48     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3140000  |
| mean 100 episode reward | -0.2     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3420000  |
| mean 100 episode reward | -0.4     |
| steps                   | 11947367 |
| success rate            | 0.37     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3430000  |
| mean 100 episode reward | -0.1     |
| steps                   | 11985014 |
| success rate            | 0.55     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3440000  |
| mean 100 episode reward | -0.2     |
| steps                   | 12022499 |
| success rate            | 0.47     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3450000  |
| mean 100 episode reward | -0.1     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3730000  |
| mean 100 episode reward | -0.2     |
| steps                   | 13169324 |
| success rate            | 0.48     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3740000  |
| mean 100 episode reward | -0.4     |
| steps                   | 13212210 |
| success rate            | 0.36     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3750000  |
| mean 100 episode reward | -0.3     |
| steps                   | 13250896 |
| success rate            | 0.44     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3760000  |
| mean 100 episode reward | -0.2     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4040000  |
| mean 100 episode reward | -0.2     |
| steps                   | 14383950 |
| success rate            | 0.46     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4050000  |
| mean 100 episode reward | -0.1     |
| steps                   | 14421091 |
| success rate            | 0.49     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4060000  |
| mean 100 episode reward | -0.2     |
| steps                   | 14458437 |
| success rate            | 0.41     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4070000  |
| mean 100 episode reward | -0.3     |
| steps                  

#### Testing

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/test_df_2800000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/success_df_2800000.csv', index=False)